In [1]:
import argparse
from typing import Any, List, Dict, Tuple, Callable

from GridWorld_environments import Grid_World
from RL_agents import ValueIterationAgent, QLearningAgent

from sklearn.preprocessing import MinMaxScaler

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from copy import deepcopy

import os
import pickle

### Value Function functionality

Functions implemented:

- `perform_value_evaluation`:
    - **Inputs**:
        - gw_env: Grid_World
        - policy: Dict[Any, Any]
    - **Outputs**:
        - Value function of the given policy
        
- `train_value_iteration`:
    - **Summary**:
        - Perform value iteration to find the optimal policy and it's value function. If `policy` or `value_function` arguemnts are used, this is will be used as a "head-start" on the search for optimality.
    - **Inputs**:
        - gw_env: Grid_World
        - policy: Dict[Any, Any]
        - value_function: Dict[Any, float]
    - **Outpus**:
        - (Policy and value function) that are optimal in the given environment.
        

#### Perform Value Evaluation function

In [2]:
def perform_value_evaluation(gw_env: Grid_World, policy: Dict[Any, Any], verbose=False):
    
    vi_agent = ValueIterationAgent(states=gw_env.get_state_space(),
                                   terminal_states=gw_env.get_terminal_states(),
                                   reward_function=gw_env.get_reward_func(),
                                   actions=gw_env.get_action_space(),
                                   gamma=GAMMA)
    
    while not vi_agent.value_converged:
        
        for state in gw_env.get_state_space():
            
            if state in gw_env.get_terminal_states():
                continue
                
            policy_act = policy[state]
            next_state = gw_env.get_new_state_on_action(old_state=state, action=policy_act)
            next_state_value = vi_agent.get_state_value(state=next_state)
            
            vi_agent.set_state_value(state=state, new_value=(gw_env.get_state_reward(state=next_state) + GAMMA * next_state_value))
        
    if verbose:
        gw_env.display_value_function(value_func=vi_agent.get_value_function())

    return vi_agent.get_value_function()

#### Train Value Iteration function (BIRL version)

In [49]:
def train_value_iteration(gw_env: Grid_World, 
                          policy: Dict[Any, Any] = None,
                          value_function: Dict[Any, float] = None,
                          verbose=False,
                          gamma=0.95,
                          training_iterations=1000
                         ) -> Tuple[Dict[Any, Any], Dict[Any, float]]:
    
    vi_agent = ValueIterationAgent(states=gw_env.get_state_space(),
                                   terminal_states=gw_env.get_terminal_states(),
                                   reward_function=gw_env.get_reward_func(),
                                   actions=gw_env.get_action_space(),
                                   gamma=gamma)

    if policy and value_function:
        raise ValueError("Can't pass both policy and value_function arguments at the same time.")
    
    if policy:
        if verbose:
            print("Calculating value function of the given policy via value evaluation...")
        vi_agent.set_value_function(new_value_function=perform_value_evaluation(gw_env, policy, verbose))
        
    if value_function:
        if verbose:
            print("Using the given value function...")
        vi_agent.set_value_function(new_value_function=value_function)
    
    iters = 0
    while iters < training_iterations and not vi_agent.value_converged:

        for state in gw_env.get_state_space():

            if state in gw_env.get_terminal_states():
                continue

            opt_act = vi_agent.get_optimal_action(action_state_pairs=gw_env.get_action_state_pairs(state=state))
            next_state = gw_env.get_new_state_on_action(old_state=state, action=opt_act)
            next_state_value = vi_agent.get_state_value(state=next_state)

            vi_agent.set_state_value(state=state, new_value=(gw_env.get_state_reward(state=next_state) + gamma * next_state_value))

        iters += 1

    if verbose:
        gw_env.display_value_function(value_func=vi_agent.get_value_function())

    vi_agent.construct_greedy_policy(gw_env.get_action_state_pairs)

    if verbose:
        gw_env.display_policy(policy=vi_agent.get_policy())

    return vi_agent.get_policy(), vi_agent.get_value_function()


### Q Function functionality

Functions implemented:

- `perform_q_function_evaluation`:
    - **Inputs**:
        - gw_env: Grid_World
        - policy: Dict[Any, Any]
    - **Outputs**:
        - Q function of the given policy
        
- `train_q_learning`:
    - **Summary**:
        - Use Q-Learning to find the optimal policy and it's Q function.
    - **Inputs**:
        - gw_env: Grid_World
    - **Outpus**:
        - (Policy and Q function) that are optimal in the given environment.
        

In [4]:
def perform_q_function_evaluation(gw_env: Grid_World, policy: Dict[Any, Any], rounding: int = 3):

    value_func = perform_value_evaluation(gw_env, policy)
    rewards = gw_env.get_reward_func()

    q_function = {}
    for state in gw_env.get_state_space():

        action_values = {}
        for action in gw_env.get_action_space():

            next_state = gw_env.get_new_state_on_action(old_state=state, action=action)
            action_values[action] = np.round(rewards[next_state] + GAMMA * value_func[next_state], rounding)
            
        q_function[state] = action_values

    return q_function
    

#### Train Q Learning function

In [5]:
def train_q_learning(gw_env: Grid_World, n_episodes=1000, verbose=False, policy="eps_greedy", eps=0.2, max_episode_len=100, gamma=0.95):
    ql_agent = QLearningAgent(states=gw_env.get_state_space(),
                              terminal_states=gw_env.get_terminal_states(),
                              reward_function=gw_env.get_reward_func(),
                              actions=gw_env.get_action_space(),
                              gamma=gamma)
    
    # init episodes
    episodes = []
    
    # Define state_space without terminal states for getting starting position
    state_space = deepcopy(gw_env.get_state_space()) # all states
    terminal_states = gw_env.get_terminal_states()
    for terminal_state in terminal_states:
        state_space.remove(terminal_state) # not non absorbing state_space
    
    for n in range(n_episodes):
        
        episode = []
        
        # random starting position
        start_idx = (np.random.choice(len(state_space)))
        start = state_space[start_idx]
        
        episode.append(start)
        
        i = 0
        terminal = False
        
        while ( ( i < max_episode_len ) and ( not terminal ) ):
            i += 1
            
            # Choose Action from S derived by given policy
            if policy == "eps_greedy":
                if np.random.uniform() < (1-eps):
                    # Choose greedy action -> highest Q-Value
                    chosen_action = ql_agent.get_greedy_action(episode[-1])
                else:
                    # Choose random action form action space
                    action_space = gw_env.get_action_space()
                    chosen_action = action_space[np.random.choice(len(action_space))]
            
            new_state = gw_env.get_new_state_on_action(episode[-1], chosen_action)
            
            # Reward is taken from Q_learning agent -> it knows the reward function from the environment
            ql_agent.update_Q_value(episode[-1], new_state, chosen_action)
            
            episode.append(new_state)
            
            if new_state in terminal_states:
                terminal = True
        
        episodes.append(episode)
    
    if verbose:
        gw_env.display_q_function(q_func=ql_agent.get_Q_function())

    ql_agent.construct_greedy_policy(gw_env.get_action_state_pairs)

    if verbose:
        gw_env.display_policy(policy=ql_agent.get_policy())

    return ql_agent.get_policy(), ql_agent.get_Q_function()


### BIRL functionality

#### Reward Space functionality

In [6]:
def get_random_reward(size: Tuple[int], r_max: float = 1.0, rounding: int = 3) -> np.ndarray:
        
    return np.random.uniform(low=-abs(r_max), high=abs(r_max), size=size).round(rounding)

def get_reward_neighbour(reward: np.ndarray, step_size: float, r_max: float = 1.0) -> np.ndarray:
    
    movement = np.random.randint(low=-1, high=2, size=reward.shape) # random array of -1, 0 and +1
    
    return (reward + step_size * movement).clip(min=-abs(r_max), max=abs(r_max)) # new neighbour reward

In [7]:
get_reward_neighbour(np.array([1, 1, 1, 1, 1, 1]), 0.2)

array([1. , 0.8, 1. , 1. , 0.8, 1. ])

#### Prior, Evidence and Posterior Distribution

In [8]:
def improper_prior(reward):
    return 1

def compute_log_posterior(gw_env:Grid_World,
                          observations: List[List[Tuple[Any]]],
                          reward: np.ndarray,
                          policy: Dict[Any, Any],
                          prior: Callable,
                          alpha: float,
                          gamma: float
                         ) -> float:
    
    gw_env.set_board(new_board=reward)
    q = perform_q_function_evaluation(gw_env=gw_env, policy=policy)
    
    log_p = 0
    
    for observation in observations:
        log_p += np.sum([alpha * q[s][a] - np.log(np.sum(np.exp(alpha * np.array(list(q[s].values()))))) for s, a in observation])
        #### Should we use np.mean() here? our obs are not all of the same length
        # log_p += np.mean([alpha * q[s][a] - np.log(np.sum(np.exp(alpha * np.array(list(q[s].values()))))) for s, a in observation])
    
    log_p += np.log(prior(reward))
    
    return log_p

def posterior_dist_ratio(gw_env: Grid_World,
                         observations: List[List[Tuple[Any]]],
                         reward_next: np.ndarray,
                         policy_next: Dict[Any, Any],
                         reward_curr: np.ndarray,
                         policy_curr: Dict[Any, Any]
                        ) -> float:
    
    prior = improper_prior
    alpha = 10
    gamma = GAMMA
    
    log_post_next = compute_log_posterior(gw_env, observations, reward_next, policy_next, prior, alpha, gamma)
    log_post_curr = compute_log_posterior(gw_env, observations, reward_curr, policy_curr, prior, alpha, gamma)
    
    return np.exp(log_post_next - log_post_curr)


#### PolicyWalk sampling algorithm

Credits for helping to decypher the formulation of the paper go to...

- https://github.com/uidilr/bayesian_irl

In [9]:
def policyWalk(observations: List[List[Tuple[Any]]],
               gw_env: Grid_World,
               step_size: float,
               n_steps: int,
               samples_burn_in: int = 500,
               samples_n_out: int = 5
              ) -> np.ndarray:
    
    # step 1: pick a random reward from the reward space (as np.array)
    reward_0 = get_random_reward(size=gw_env.get_board().shape)
    
    # step 2: get optimal policy given reward_0
    gw_env.set_board(new_board=reward_0)  # board is the np.array representation of the reward function
    policy_0, val_func_0 = train_value_iteration(gw_env=gw_env)
    
    posterior_samples = []
    
    # step 3: explore the reward space and sample from the posterior (GridWalk as a MCMC (Metropolis algorithm))
    for i in range(n_steps):
        # step 3a: pick reward_1 from reward_0 neighbours randomly (step_size distance)
        reward_1 = get_reward_neighbour(reward=reward_0, step_size=step_size)
        
        # step 3b: compute the Q function of reward_1 under policy_0
        gw_env.set_board(new_board=reward_1)
        q_func = perform_q_function_evaluation(gw_env=gw_env,
                                               policy=policy_0)
        
        # step 3c: check if ... 
        if is_policy_not_optimal(q_func=q_func, policy=policy_0,
                                 states=gw_env.get_state_space(), actions=gw_env.get_action_space()):
            # step 3c.i: get optimal policy given reward_1 AND policy_0 (we can use the value function of policy_0 directly)
            #### MOTIVATION: since reward_1 is a neighbour of reward_0, policy_1 will be close to policy_0. We use this fact to speed up the computation
            #### would be a repetition of step 3b: gw_env.set_board(new_board=reward_1)
            policy_1, val_func_1 = train_value_iteration(gw_env=gw_env,
                                                         value_function=val_func_0)
            
            # step 3c.ii: update reward_0 AND policy_0 with prob. according to the posterior_dist
            if np.random.uniform() < np.minimum(1, posterior_dist_ratio(gw_env, observations, reward_1, policy_1, reward_0, policy_0)):
                reward_0 = deepcopy(reward_1)
                policy_0 = deepcopy(policy_1)
                val_func_0 = deepcopy(val_func_1)
            
        else:
            # step 3c.iii: update reward_0 with prob. according to the posterior_dist
            if np.random.uniform() < np.minimum(1, posterior_dist_ratio(gw_env, observations, reward_1, policy_0, reward_0, policy_0)):
                reward_0 = deepcopy(reward_1)
                
        if (i > samples_burn_in) and (samples_n_out != 0) and ((i - samples_burn_in) % samples_n_out == 0):
            posterior_samples.append(reward_0)

    # step 4: return a sample from the posterior
    if samples_n_out != 0:
        return posterior_samples
    else:
        return reward_0

def is_policy_not_optimal(q_func, policy, states, actions):
    for state in states:
        for action in actions:
            if q_func[state][policy[state]] < q_func[state][action]:
                return True
    return False


### Step-by-step algorithm

In [51]:
GAMMA = 0.95

VALUE_ITERATION_TRAINING_N = 1000

GW_SIZE = (7, 7)
GW_TRAPS = []
GW_GOALS = [(0, 0)]

NUMBER_OF_STATES = GW_SIZE[0] * GW_SIZE[1]

NUMBER_OF_TRAJECTORIES    = NUMBER_OF_STATES * 20
MAXIMUM_TRAJECTORY_LENGTH = NUMBER_OF_STATES * 4

BIRL_N_OF_POSTERIOR_SAMPLES     = 10
BIRL_POLICYWALK_STEP_SIZE       = 0.05
BIRL_POLICYWALK_N_STEPS         = int(NUMBER_OF_STATES / BIRL_POLICYWALK_STEP_SIZE * 20)
BIRL_POLICYWALK_SAMPLES_BURN_IN = int(BIRL_POLICYWALK_N_STEPS * 0.7)
BIRL_POLICYWALK_SAMPLES_N_OUT   = 10


In [52]:
environment = Grid_World(size=GW_SIZE, traps=GW_TRAPS, goals=GW_GOALS, randomize_board=False)

target_reward = environment.get_board()

vi_greedy_policy, vi_greedy_value_function = train_value_iteration(gw_env=environment, verbose=True, gamma=GAMMA, training_iterations=VALUE_ITERATION_TRAINING_N)

print(f"Generating {NUMBER_OF_TRAJECTORIES} trajectories...")

trajectories = environment.generate_trajectories(policy=vi_greedy_policy,
                                                 n_traj=NUMBER_OF_TRAJECTORIES,
                                                 max_traj_length=MAXIMUM_TRAJECTORY_LENGTH,
                                                 return_state_action_pairs=True)


Value function:
[[0.         1.         0.95       0.9025     0.857375   0.81450625
  0.77378094]
 [1.         0.95       0.9025     0.857375   0.81450625 0.77378094
  0.73509189]
 [0.95       0.9025     0.857375   0.81450625 0.77378094 0.73509189
  0.6983373 ]
 [0.9025     0.857375   0.81450625 0.77378094 0.73509189 0.6983373
  0.66342043]
 [0.857375   0.81450625 0.77378094 0.73509189 0.6983373  0.66342043
  0.63024941]
 [0.81450625 0.77378094 0.73509189 0.6983373  0.66342043 0.63024941
  0.59873694]
 [0.77378094 0.73509189 0.6983373  0.66342043 0.63024941 0.59873694
  0.56880009]]
Policy:
[['x' '<' '<' '<' '<' '<' '<']
 ['^' '<' '<' '<' '<' '<' '<']
 ['^' '<' '<' '<' '<' '<' '<']
 ['^' '<' '<' '<' '<' '<' '<']
 ['^' '<' '<' '<' '<' '<' '<']
 ['^' '<' '<' '<' '<' '<' '<']
 ['^' '<' '<' '<' '<' '<' '<']]
Generating 980 trajectories...


In [ ]:
print(f"Generate {BIRL_N_OF_POSTERIOR_SAMPLES} samples from the posterior")

posterior_samples = []

for _ in tqdm(range(BIRL_N_OF_POSTERIOR_SAMPLES)):
    
    # restart the environment
    environment = Grid_World(size=GW_SIZE, traps=GW_TRAPS, goals=GW_GOALS, randomize_board=False)

    posterior_samples += policyWalk(observations=trajectories,
                                    gw_env=environment,
                                    step_size=BIRL_POLICYWALK_STEP_SIZE,
                                    n_steps=BIRL_POLICYWALK_N_STEPS,
                                    samples_burn_in=BIRL_POLICYWALK_SAMPLES_BURN_IN,
                                    samples_n_out=BIRL_POLICYWALK_SAMPLES_N_OUT)

Generate 10 samples from the posterior


  0%|                                                                                                                               | 0/10 [00:00<?, ?it/s]/var/folders/s7/n_rwl6bn6_q6_wbbp5xx_n3r0000gn/T/ipykernel_22545/2626929354.py:42: RuntimeWarning: overflow encountered in exp
  return np.exp(log_post_next - log_post_curr)


In [41]:
posterior_samples

[array([[ 0.95 , -0.85 , -0.95 , -0.9  , -0.65 ],
        [ 0.123, -1.   , -1.   , -0.6  , -0.8  ],
        [-0.437, -1.   , -0.6  , -0.45 , -0.75 ],
        [-0.45 , -0.525, -0.65 , -0.75 , -1.   ]]),
 array([[ 0.95 , -0.85 , -0.95 , -0.9  , -0.65 ],
        [ 0.123, -1.   , -1.   , -0.6  , -0.8  ],
        [-0.437, -1.   , -0.6  , -0.45 , -0.75 ],
        [-0.45 , -0.525, -0.65 , -0.75 , -1.   ]]),
 array([[ 0.95 , -0.85 , -0.95 , -0.9  , -0.65 ],
        [ 0.123, -1.   , -1.   , -0.6  , -0.8  ],
        [-0.437, -1.   , -0.6  , -0.45 , -0.75 ],
        [-0.45 , -0.525, -0.65 , -0.75 , -1.   ]]),
 array([[ 0.95 , -0.85 , -0.95 , -0.9  , -0.65 ],
        [ 0.123, -1.   , -1.   , -0.6  , -0.8  ],
        [-0.437, -1.   , -0.6  , -0.45 , -0.75 ],
        [-0.45 , -0.525, -0.65 , -0.75 , -1.   ]]),
 array([[ 0.9  , -0.9  , -0.9  , -0.9  , -0.6  ],
        [ 0.123, -1.   , -1.   , -0.6  , -0.8  ],
        [-0.437, -1.   , -0.55 , -0.5  , -0.75 ],
        [-0.45 , -0.475, -0.6  , -0.7  , -

In [42]:
np.mean(posterior_samples, axis=0)

array([[ 0.5366841 , -0.85213389, -0.88560669, -0.86552301, -0.77543933],
       [ 0.09633515, -0.97606695, -0.94797071, -0.82889121, -0.81910042],
       [-0.50530042, -0.93774059, -0.71374477, -0.80786611, -0.81466527],
       [-0.48515063, -0.57995565, -0.72698745, -0.75606695, -0.8158159 ]])

In [43]:
len(posterior_samples)

2390

#### Store the data

In [44]:
dict_to_store = {
    "metadata": {
        "algorithm": "BIRL - Value Iteration",
        "environment": "Grid_World",
        "env_n_of_states": NUMBER_OF_STATES,
        "env_size": GW_SIZE,
        "env_traps": GW_TRAPS,
        "env_goals": GW_GOALS,
        "gamma": GAMMA,
        "expert_n_of_trajs": NUMBER_OF_TRAJECTORIES,
        "expert_max_traj_length": MAXIMUM_TRAJECTORY_LENGTH,
        "BIRL_n_of_iters": BIRL_N_OF_POSTERIOR_SAMPLES,
        "BIRL_step_size": BIRL_POLICYWALK_STEP_SIZE,
        "BIRL_n_steps": BIRL_POLICYWALK_N_STEPS,
        "BIRL_burn_in": BIRL_POLICYWALK_SAMPLES_BURN_IN,
        "BIRL_n_out": BIRL_POLICYWALK_SAMPLES_N_OUT
    },
    "data": {
        "target_reward": target_reward,
        "expert_greedy_policy": vi_greedy_policy,
        "expert_greedy_val_func" : vi_greedy_value_function,
        "predicted_rewards": posterior_samples
    }
}

In [45]:
dict_to_store

{'metadata': {'algorithm': 'BIRL - Value Iteration',
  'environment': 'Grid_World',
  'env_n_of_states': 20,
  'env_size': (4, 5),
  'env_traps': [],
  'env_goals': [(0, 0)],
  'gamma': 0.95,
  'expert_n_of_trajs': 400,
  'expert_max_traj_length': 80,
  'BIRL_n_of_iters': 10,
  'BIRL_step_size': 0.05,
  'BIRL_n_steps': 8000,
  'BIRL_burn_in': 5600,
  'BIRL_n_out': 10},
 'data': {'target_reward': array([[1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]),
  'expert_greedy_policy': {(0, 0): (1, 0),
   (0, 1): (0, -1),
   (0, 2): (0, -1),
   (0, 3): (0, -1),
   (0, 4): (0, -1),
   (1, 0): (-1, 0),
   (1, 1): (0, -1),
   (1, 2): (0, -1),
   (1, 3): (0, -1),
   (1, 4): (0, -1),
   (2, 0): (-1, 0),
   (2, 1): (0, -1),
   (2, 2): (0, -1),
   (2, 3): (0, -1),
   (2, 4): (0, -1),
   (3, 0): (-1, 0),
   (3, 1): (0, -1),
   (3, 2): (0, -1),
   (3, 3): (0, -1),
   (3, 4): (0, -1)},
  'expert_greedy_val_func': {(0, 0): 0.0,
   (0, 1): 

In [46]:
BIRL_file_suffix = "BIRL_"

# find largest file number of BIRL file in "data" folder
largest_file_number = 0

for file in os.listdir("data"):
    if BIRL_file_suffix in file:
        after_suffix = (file[len(BIRL_file_suffix):])
        file_number = int(after_suffix.split(".")[0])
        if file_number > largest_file_number:
            largest_file_number = file_number

print(f"Found largest file number of BIRL data file: {largest_file_number}")


print(f"Writing next BIRL data file with number: {largest_file_number + 1}")
# write dict to next file
with open(os.path.join("data", f"{BIRL_file_suffix}{largest_file_number + 1}.pkl"), "wb") as file:
    pickle.dump(dict_to_store, file)

Found largest file number of BIRL data file: 1
Writing next BIRL data file with number: 2


In [47]:
with open(os.path.join("data", f"{BIRL_file_suffix}{largest_file_number + 1}.pkl"), 'rb') as file:
    
    loaded_dict = pickle.load(file)

In [48]:
np.mean(loaded_dict["data"]["predicted_rewards"], axis=0)

array([[ 0.5366841 , -0.85213389, -0.88560669, -0.86552301, -0.77543933],
       [ 0.09633515, -0.97606695, -0.94797071, -0.82889121, -0.81910042],
       [-0.50530042, -0.93774059, -0.71374477, -0.80786611, -0.81466527],
       [-0.48515063, -0.57995565, -0.72698745, -0.75606695, -0.8158159 ]])

In [ ]:
loaded_dict["data"]["target_reward"]